In [ ]:
shared_data_dir = "unset"
dagrun_data_dir = "unset"
admin_file_path = "admin3.geojson"
admin_level = "admin3"
names_table = "[]"

In [ ]:
import flowclient as fc
from pathlib import Path
from os import environ as container_environ
import json
import pandas as pd
import geopandas as gpd

fc_conn = fc.connect(
    url=container_environ["FLOWAPI_URL"],
    ssl_certificate=False,  # Workaround pending https://github.com/Flowminder/flowpyter-task/issues/35
    token=container_environ["FLOWAPI_TOKEN"],
)
geog = fc.get_geography(connection=fc_conn, aggregation_unit=admin_level)

In [ ]:
geog_names = pd.DataFrame.from_records(
    eval(names_table), columns=["admin3pcod", "admin3name", "admin2name", "admin1name"]
)
geog_names = (
    geog_names.drop_duplicates()
)  # Need this for fix for Airflow SQL Xcom issue
geog_names

In [ ]:
geog_table = gpd.GeoDataFrame.from_features(geog)
geog_table = geog_table.join(geog_names.set_index("admin3pcod"), how="left", on="pcod")
geog_table = geog_table.rename(columns={"pcod": "admin3pcod"})
geog_table = geog_table.drop_duplicates(subset=["admin3pcod"])
geog_table

In [ ]:
geog_path = Path(dagrun_data_dir) / admin_file_path
geog_table.to_file(geog_path, driver="GeoJSON")